In [1]:
import Adafruit_SSD1306
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

from jetbot.utils.utils import get_ip_address
import subprocess
import time

In [2]:
# OLED-SSD1306 초기화

# 하드웨어 I2C 방식의 128x32 Display
# gpio는 인식문제 해결을 위해 1로 설정
disp = Adafruit_SSD1306.SSD1306_128_32(rst=None,i2c_bus=0,gpio=1)

# 디스플레이 초기화
disp.begin()

# 화면 클리어
disp.clear()
disp.display()

# 디스플레이 이미지 생성, 흑백(1-bit) 이미지 사용
disp_image = Image.new('1', (disp.width, disp.height))

# 이미지에 그릴 드로잉 객체 얻기
screen = ImageDraw.Draw(disp_image)

# 화면 전체를 검은색으로 채워서 지우기
screen.rectangle((0,0, disp.width, disp.height), outline=0,fill=0)

# 화면의 패딩영역 위치 조정 
screen_padding = -2
screen_top = screen_padding
screen_botton = disp.height - screen_padding
screen_x = 0

# 기본 서체 사용
font = ImageFont.load_default()

In [3]:
# OLED 화면에 기본 정보 디스플레이

### OLED 화면에 시스텐 정보 4줄을 보여주는 함수

## 로컬 IP 얻기
def getip():
    return str(get_ip_address('wlan0'))

def ShowInfo(strInfo):
    # 화면 전체를 검은색으로 채워서 지우기
    screen.rectangle((0,0, disp.width, disp.height), outline=0,fill=0)
    
    # 화면 맨 위의 첫 번째 줄에는 매개변수로 넘겨받을 문자열을 디스플레이
    screen.text((screen_x,screen_top+0), strInfo, font=font, fill=255)
    
    # 두 번째 줄에는 젯봇의 ip address를 디스플레이
    screen.text((screen_x,screen_top+8), "IP: " + getip(), font=font, fill=255)
    
    # 세 번째 줄에는 메모리 사용 상태를 디스플레이
    cmd = "free -m | awk 'NR=={printf \"Mem:%s/%sM %.2f%%\", $3,$2,$3*100/$2}'"
    MemUsage = subprocess.check_output(cmd,shell = True)
    
    # 네 번째 줄에는 디스크 사용 상태를 디스플레이
    cmd = "df -h | awk 'NF==\"/\"{printf \"Disk:%d/%dGB %s\", $3,$2,$5}'"
    Disk = subprocess.check_output(cmd,shell = True)
    screen.text((screen_x,screen_top+25), str(Disk.decode('utf-8')), font=font, fill=255)
    
    # 설정한 내용을 OLED 화면에 출력
    disp.image(disp_image)
    disp.display()

In [4]:
### 맨 위의 1번쨰 줄에 텍스트를 디스플레이 하는 함수

def ShowTopInfo(strInfo):
    screen.rectangle((0,0, disp.width,7), outline=0,fill=0)
    screen.text((screen_x,screen_top), strInfo, font=font, fill=255)
    
    # 설정한 내용을 OLED 화면에 출력
    disp.image(disp_image)
    disp.display()
    
ShowTopInfo("Home Security Monitor")

In [5]:
### 카메라 핀/틸트 서보 초기화와 사용을 위한 모듈 임포트

from servoserial import ServoSerial
import time

### io핀 권한 획득
import os
os.system("sudo chmod 777 /dev/ttyTHS1")  # 여기서 권한을 획득하면 잘 안되는 경우가 있어서 터미널에서 해주는 것이 좋음

256

In [7]:

pantilt = ServoSerial()

# 카메라 서보 팬/틸트 회전 가능 최댓값
MAX_PAN = 4096  # 좌우 Pan 범위 : 0(오른쪽 끝) ~ 4095(왼쪽 끝)
MAX_TILT = 4096  # 상하 Pan 범위 : 0(아래쪽 끝) ~ 4095(위쪽 끝)

# 카메라 서보 팬/틸트 오차 조정값 (상하좌우 원점 조정)
PAN_ADJ = 212  # Camera_Pan_Servor_Adject = "212"
TILT_ADJ = 0  # Camera_Tilt_servor_Adject = "0

# 오차 조정값을 적용하여 자주 사용할 팬/틸트의 중앙값 설정, 중앙에 있어서 얘를 제어하여 위치 조절
MID_PAN = int(MAX_PAN/2 + PAN_ADJ)
MID_TILT = int(MAX_TILT/2 + TILT_ADJ)

# 초기 카메라의 상하 틸트 위치 (약간 위쪽을 비리보도록 설정)
DEF_TILT = MID_TILT + 100  # 중앙 상향 사람인식 팬/틸트 위치

CurrentPan = MID_PAN  # 현제 서보 pan 위치값
CurrentTILT = DEF_TILT  # 현제 서보 tilt 위치값

serial Open!


In [8]:
# 팬/틸트 각도를 제어하는 함수 
def MovePanTilt(pan,tilt):  # 매개변수로 넘겨받은 pan과 tilt값을 절대위치 설정 값으로 적용하여 제어
    global CurrentPan
    global CurrentTILT
    
    if (0<= pan) and (pan<MAX_PAN):
        CurrentPan = pan
    if (0<=tilt) and (tilt<MAX_TILT):
        CurrentTILT = tilt
        
    pantilt.Servo_serial_double_control(1,CurrentPan,2,CurrentTILT)
    

# 팬/틸트 각도를 제어하는 함수 
def MoveRelativePanTilt(pan,tilt):  # 매개변수로 넘겨받은 pan과 tilt값을 상대위치 설정 값으로 적용하여 제어
    global CurrentPan
    global CurrentTILT
    
    if (0<= CurrentPan + pan) and (CurrentPan + pan < MAX_PAN):
        CurrentPan += pan
    if (0<= CurrentTILT + tilt) and (CurrentTILT + tilt < MAX_TILT):
        CurrentTILT += tilt
        
    pantilt.Servo_serial_double_control(1,CurrentPan,2,CurrentTILT)

In [9]:
### RGB 스트립 제어
from RGB_Lib import Programing_RGB

RGB = Programing_RGB()

RGB.OFF_ALL_RGB()  # 끄기

In [11]:
### I/O 제어 : 부저버튼 초기화
import RPi.GPIO as gpio
gpio.setwarnings(False)

BEEP_pin = 6  # buzzer 핀 번호

KEY1_pin = 8  # BOARD pin87, k1버튼
KEY2_pin = 7  # BOARD pin 7, k2버튼

gpio.setmode(gpio.BCM)  # 보드 핀 번호 양식

gpio.setup(BEEP_pin, gpio.OUT, initial = gpio.LOW)  # 초기상태 끄기

gpio.setup(KEY1_pin, gpio.IN)  # KEY1 버튼 pin set as input
gpio.setup(KEY2_pin, gpio.IN)  # KEY2 버튼 pin set as input

def Beep(beepTime):
    
    gpio.output(BEEP_pin, gpio.HIGH)  # 부저 켜기
    time.sleep(beepTime)  # 소리내는 시간 
    gpio.output(BEEP_pin, gpio.LOW)  # 부저 끄기

## 카메라 추적 자동/수동 모드 (AutoDetectAndTrace)

### 자동추적(AutoDetectAndTrace=1) 모드일 때:

    * 카메라 입력 영상 프레임 이미지에서 사람을 검출합니다.
    * 만약 사람이 컴출되면 카메라 및 팬/틸트가 자동으로 사람을 추적하며,
    * 원격제어 APP으로 사람이 검출되었음을 알립니다.
    
### 수동추적(AutoDetectAndTrace=0) 모드일 때:
    
    * 카메라 입력 영상 프레임 이미지에서 사람을 검출하지 않습니다.
    * 원격제어 APP으로 카메라 팬/틸트를 조종하며 실시간 영상을 볼 수 있습니다.
    * 원격제어 APP으로 알림을 켜거나 끌 수 있습니다.
    
처음에 자동추적 모드로 시작하여 사람이 검출되면 원격제어 APP으로 사람이 검출되었음을 알립니다.

원격제어 APP에서 "수동모드" 버튼을 눌러서 수동제어 모드로 전환하거나,
원격제어 APP에서 "지동모드" 버튼을 눌러서 지동제어 모드로 전환할 수 있습니다.

In [10]:
AutoDetectAndTrace = 1  # 자동추적(AutoDetectAndTrace=1) 모드로 시작됨

In [11]:
### 알람 울리기 끄기 스레드
from threading import Thread

DoAlarm = 0  # 알람 울리기(=1), 알람 끄기(=0), 원격 제어 APP에서 선택 가능 
LastDoAlarm = DoAlarm  # 알람이 켜져 있다가 앱에서  알람을 끄면 모든 LED를 끄기 위해 사용

### 알람 스레드가 실행할 코드
def DoAlarmThread():
    global DoAlarm
    print("===> DoAlarmThread() started. \n");
    
    while Tr:
        if DoAlarm:  # 알람 상태이면 1초마다 부저소리와 함꼐 LED를 빨강과 파랑으로 번갈아 켭니다.
            LastDoAlarm = DoAlarm
            RGB.Set_All_RGB(255,0,0)  # 빨강으로 켜기
            if DoAlarm:  # 아직도 알람이 켜진 상태라면 계속 진행, 아니면 LED 끄기로
                Beep(1)
                if DoAlarm:  # 아직도 알람이 켜진 상태라면 계속 진행, 아니면 LED 끄기로
                    RGB.Set_All_RGB(0,0,255)  # 파랑으로 켜기
        else:
            if LastDoAlarm:  # 직전에 알람이 켜져 있었으면 모든 LED 끄기
                LastDoAlarm = DoAlarm
                RGB.OFF_ALL_RGB()
                
        time.sleep(1)
        
### 알람 스레드를 시작합니다
def StartDoAlarmThread():
    thread = Thread(target = DoAlarmThread, daemon = True).start()  # 프로그램 종료시 알람 스레드도 함계 종료                

In [12]:
### 검출된 위치에 따라 팬/틸트 방향을 제어하기 위한 PID 제어 초기화
import PID

pid_pan = PID.PositionalPID(1.9,0.3,0.35)
pid_tilt = PID.PositionalPID(1.5,0.2,0.3)

In [13]:
### 카메라 인스턴스 생성 및 초기화
from jetbot import Camera
from jetbot import bgr8_to_jpeg

camera = Camera.instance(width=720,height=720)

In [14]:
from flask import Flask, render_template, Response

import socket
import base64
import hashlib
import sys
import struct
import threading
import hashlib
import re

In [15]:
g_init = False  # 이 서버를 한 번만 실행하기 위해서 사용하는 flag, 웹 서버가 초기화될 때 데몬(demon)이 한 번만 실행되도록 체크 하는데 사용 
                # 데몬(demon, 서비스의 요청에 대해 응답하기 위해 오랫동안 실행중인 백그라운드(background) 프로세스)
                # 백그라운드(background) 프로세스: 콘솔이나 터미널을 이용하지 않고 스스로 동작되는 프로세스 
                # 나중에 만들 start_tcp_server() 함수 안에서 g_init 값을 true로 설정하면 init 함수 안에서 g_init 값이 
                # 이미 True인 경우 웹소켓 서버 스레드를 만들지 않고 바로 끝을 낸다
                # 웹소켓 : 서버와 클라이언트 간에 Socket Connection을 유지해서 언제든 양방향 통신 또는 데이터 전송이 가능하도록 하는 기술
                # Real-time web application구현을 위해 널리 사용되어지고 있다
                # Server 에서 Client로 바로 Event를 줄 수 있다는 점에서 성능향 효율적이다. (polling 과 같이 불필요한 request 가 없어도 됨)
        
g_socket = None  # 사람이 검출 되었을 때 사용자 앱으로 알려줄 때 사용할 통신 소켓을 저장할 변수, 클라이언트의 요청이 와서 연결된 소켓을 저장
                 # 카메라에서 사람이 검출 되었을 떄 우너격제어 앱으로 알림을 보낼 때 
                 

#flask 인스턴스 생성
app = Flask(__name__)

# 웹 소켓 서버가 클라이언트에 전송하는 내용으로 {1}과 {2}의 내용은 나중에 바뀝니다.
# 클라이언트가 웨ㅔㅂ소켓 서버로 연결을 요청하는 작업을 핸드셰이킹이라고 한다.
# 아래 코드는 서버가 클라이언트인 원격제어 엡으로 전송하는 내용의 템플릿
HANDSHACK_STRING = "HTTP/1.1 101 Switching Protocols/r/n" \
    "Upgrade:websocket\r\n" \
    "connection: Upgrade\r\n" \
    "Sec-webSocket-Accept: {1}\r\n" \  # {1}에는 응답-키 스트링이 들어감
    "WebSocket-Location: ws://{2}/chat\r\n"  \  # {2}에는 IP주소와  통신 포트 번호를 대입
    "WebSoket-Protocol:chat\r\n\r\n"

In [16]:
### 클라이언트 웹 브라우저에서 경로 "/"를 요청하면 index() 함수를 실행하여 index.htmk 페이지를 보내줍니다
@app.route('/')  # 주소 경로가 '/'이면 아래의 함수를 실행하도록 매칭이 된다, 즉 최상위 폴더를 요청한다면 index가 실행되는 것, 결과적으로 index.html을 원격제어 엡에 전송한다 
def index():
    return render_template('index.html')

### 클라이언트의 웹 브라우저에서 경로 '/video_feed'를 요청하면 video_feed 함수를 실행합니다.
# mimetype= 'multipart/x-mixed-replace; boundary=frame' : 애니메이션을 웹에서 구현하기 위한 MIME타입, mode_handle에서 연속적으로 반환해주는 jpeg 이미지를 매번 화면에 업데이트함
@app.route('/video_feed')  #  
def video_feed():
    return Response(mode_handle(), mimetype= 'multipart/x-mixed-replace; boundary=frame')

* CGI : 공통 게이트웨이 인터페이스(Common Gateway Interface)의 약어로, 웹서버와 외부 프로그램 사이에서 정보를 주고받는 방법이나 규약들을 말한다.
*     : 웹서버와 요청을 받아 처리해줄 로직을 담고 있는 애플리케이션 프로그램 사이의 인터페이스
* MIME : MIME이란? Multipurpose Internet Mail Extensions의 약자, 웹을 통해서 여러형태의 파일 전달하는데 쓰임, 여러 형식이 있는데 multipart는 '복합문서' 형식을 의미 
* x-mixed-replace는 복삼문서 들 중에서도 서버밀기에 사용되는 형식이라는 의미 x-는 아직 정식으로 표준화 되지 않은 형식을 의미 boundary는 복합문서내의 문서들을 구분하는 분리자}이다

In [ ]:
### 클라이언트웹 브라우저에서 경로 '/init'을 요청하면 init() 함수를 실행합니다.
# Jetbot의 IP를 얻고, 스레드를 열어서 6000번 대 포트에서 TCP서버를 시작하는 함수를 실행
# g_init 플레그를 두어서 start_tcp_server를 한 번 만 실행시키도록 합니다.
# 서버 시작 스레드가 완료되면 init.html 페이지를 보냐줍니다.
@app.route('/init')